# **Guía Interactiva del Algoritmo Byte-Pair Encoding (BPE)**

Bienvenido/a a esta guía interactiva donde desglosaremos, paso a paso, el funcionamiento del algoritmo **Byte-Pair Encoding (BPE)**, la base de los tokenizadores modernos que utilizan los Grandes Modelos de Lenguaje (LLMs).

A lo largo de este notebook, cubriremos:
1.  **Conceptos Fundamentales:** Cómo se representa el texto en un ordenador, desde los strings de Python hasta la codificación de bytes en UTF-8.
2.  **Implementación de BPE desde Cero:** Escribiremos juntos el algoritmo BPE para entender su lógica de fusión de pares de bytes.
3.  **Codificación y Decodificación:** Veremos cómo usar nuestro tokenizador para convertir texto a tokens y viceversa.
4.  **Técnicas Avanzadas:** Exploraremos las mejoras que usan los tokenizadores reales, como la pre-tokenización con expresiones regulares (Regex) de GPT-2.
5.  **Un Vistazo al Ecosistema:** Analizaremos brevemente otras herramientas estándar de la industria como `tiktoken` de OpenAI y `SentencePiece` de Google.

## **1. La Base: De Texto a Bytes**

Antes de poder tokenizar, debemos entender cómo un ordenador "ve" el texto. Un modelo de lenguaje no opera con letras, sino con números.

### **1.1. Strings en Python: Secuencias de Puntos de Código Unicode**

Cuando escribes un [`str`](https://docs.python.org/3/library/stdtypes.html#text-sequence-type-str) en Python, como `"hola 👋"`, estás creando una secuencia de **puntos de código [Unicode](https://es.wikipedia.org/wiki/Unicode)**. Unicode es un estándar universal que asigna un número único a (casi) cualquier carácter o símbolo que puedas imaginar.

Podemos ver estos números con la función `ord()`:

In [3]:
ord("h")

104

In [4]:
[ord(x) for x in "Hello World"]

[72, 101, 108, 108, 111, 32, 87, 111, 114, 108, 100]

In [5]:
ord("😃")

128515

### **1.2. De Unicode a Bytes: La Codificación UTF-8**

Los puntos de código Unicode son abstractos. Para almacenar o transmitir texto, necesitamos codificarlos en **bytes**, que son la unidad fundamental de información en un ordenador (un número entre 0 y 255).

La codificación más común es **UTF-8**, un sistema de longitud variable muy eficiente. Por ejemplo:
*   Caracteres [ASCII](https://elcodigoascii.com.ar/) (inglés sin acentos) usan 1 byte.
*   Letras con tildes (como 'á') usan 2 bytes.
*   Emojis y símbolos más complejos (€) pueden usar 3 o 4 bytes.

En Python, convertimos un `str` a bytes con el método `.encode("utf8")`. El resultado es una secuencia de enteros entre 0 y 255 (que es el rango de valores que puede tomar un número de 1 byte).

In [6]:
list("Hello World".encode("utf-8"))

[72, 101, 108, 108, 111, 32, 87, 111, 114, 108, 100]

En este caso, esta cadena esta compuesta únicamente por carácteres ASCII, por lo que los números son los mismos a los vistos arriba en la cadena de `"Hello World"`, sin embargo, si vemos el valor que toma el emoji `😃` veremos que es muy diferente.

In [7]:
# Ver los bytes
raw_bytes = "😃".encode("utf8")
print(f"Bytes de 😃: {raw_bytes}")


# Representaciones numéricas
number_bytes = list("😃".encode("utf8"))
print(f"\nRepresentación numérica de 😃: {number_bytes}")

Bytes de 😃: b'\xf0\x9f\x98\x83'

Representación numérica de 😃: [240, 159, 152, 131]


Como podemos ver, este caracter es más complejo y requere de 4 bytes para representarse.

Esta representación en bytes es la materia prima sobre la que opera el algoritmo BPE. Al trabajar con bytes, nos aseguramos de que **nunca habrá un carácter "desconocido"**. Cualquier texto, en cualquier idioma, puede descomponerse en una secuencia de bytes.

## **2. El Algoritmo BPE: Paso a Paso**

El algoritmo BPE es un método de compresión de datos que busca iterativamente el par de bytes más frecuente en un texto y lo reemplaza con un nuevo byte (o, en nuestro caso, un nuevo token con un ID mayor que 255).

Vamos a implementarlo desde cero con un fragmento de *El Quijote*.

### **2.1. Nuestro Corpus de Entrenamiento**

Primero, convertimos nuestro texto a una secuencia de bytes (representados como enteros).

In [8]:
text = "En un lugar de la Mancha, de cuyo nombre no quiero acordarme, no ha mucho tiempo que vivía un hidalgo de los de lanza en astillero"
print(f"El corpus tiene {len(text)} caracteres")

text_bytes = text.encode("utf8")
print(f"El corpus tiene {len(text_bytes)} bytes")

El corpus tiene 130 caracteres
El corpus tiene 131 bytes


Hmmm... ¿Cómo puede ser que tengamos más bytes que caracteres?

Si has prestado anteción, te habrás dado cuenta que el texto contiene una tilde (en `"vivía"`) cuya representación no equivale solo a un byte.

### **2.2. Encontrar el Par Más Frecuente**

Ahora, necesitamos una función que recorra la secuencia de tokens y cuente la frecuencia de cada par adyacente, para esto utilizaremos la función `get_stats()`, que nos proporciona una lista, ordenada de mayor a menor frecuencia.

In [9]:
def get_stats(ids):
    counts = {}
    for pair in zip(ids, ids[1:]):
        counts[pair] = counts.get(pair, 0) + 1
    return counts

stats = get_stats(text_bytes)
print(sorted(((v, k) for k, v in stats.items()), reverse = True))

[(7, (111, 32)), (6, (101, 32)), (4, (110, 32)), (4, (100, 101)), (4, (97, 32)), (4, (32, 108)), (4, (32, 100)), (3, (110, 111)), (3, (32, 110)), (2, (117, 110)), (2, (116, 105)), (2, (114, 111)), (2, (113, 117)), (2, (108, 97)), (2, (105, 101)), (2, (104, 97)), (2, (101, 114)), (2, (100, 97)), (2, (99, 104)), (2, (97, 114)), (2, (97, 110)), (2, (44, 32)), (2, (32, 117)), (2, (32, 113)), (2, (32, 104)), (2, (32, 97)), (1, (195, 173)), (1, (173, 97)), (1, (122, 97)), (1, (121, 111)), (1, (118, 195)), (1, (118, 105)), (1, (117, 121)), (1, (117, 105)), (1, (117, 103)), (1, (117, 101)), (1, (117, 99)), (1, (115, 116)), (1, (115, 32)), (1, (114, 109)), (1, (114, 101)), (1, (114, 100)), (1, (114, 32)), (1, (112, 111)), (1, (111, 115)), (1, (111, 114)), (1, (111, 109)), (1, (110, 122)), (1, (110, 99)), (1, (109, 117)), (1, (109, 112)), (1, (109, 101)), (1, (109, 98)), (1, (108, 117)), (1, (108, 111)), (1, (108, 108)), (1, (108, 103)), (1, (108, 101)), (1, (105, 118)), (1, (105, 108)), (1, (10

En nuestro caso, el par que más veces aparece es el `(111, 32)`. Que podemos ver que corresponde a...

In [10]:
pair = bytes([32]).decode("utf8") + bytes([111]).decode("utf8")
print(f"El par más frecuente es: '{pair}'")

El par más frecuente es: ' o'


¡Es la letra "o" seguida de un espacio! Esto tiene sentido, ya que muchas palabras en español terminan en "o".

### **2.3. Fusionar el Par y Crear un Nuevo Token**

Ahora, reemplazamos todas las ocurrencias del par `(111, 32)` con un nuevo token. Nuestro vocabulario inicial son los 256 bytes (IDs de 0 a 255), así que el primer token nuevo tendrá el ID `256`.

In [11]:
top_pair = (111, 32)

def merge(ids, pair, idx):
    newids = list()
    i = 0
    while i < len(ids):
        if i < len(ids) - 1 and ids[i] == pair[0] and ids[i+1] == pair[1]:
            newids.append(idx)
            i += 2
        else:
            newids.append(ids[i])
            i += 1
    return newids

text_bytes = merge(text_bytes, top_pair, 256)
print(len(text_bytes))

124


La longitud de nuestra secuencia se ha reducido de 131 a 124. Esto es `131 - 7 = 124`, ¡exactamente las 7 ocurrencias del par `('o', ' ')` que fusionamos!

### **2.4. El Proceso Iterativo**

Repetimos este proceso (buscar el par más frecuente, fusionarlo y reemplazarlo) un número determinado de veces. Este número lo define el `vocab_size` que deseemos. Si queremos un vocabulario de 276 tokens, realizaremos `276 - 256 = 20` fusiones.

Vamos a automatizarlo en un bucle.

In [12]:
# --- COMENTARIOS SUGERIDOS PARA EL CÓDIGO ---
vocab_size = 276
num_merges = vocab_size - 256

# Copiamos la lista de tokens para no modificar la original
ids = list(text_bytes)

# El diccionario 'merges' almacenará las reglas de fusión aprendidas en orden
# (par) -> nuevo_id
merges = {}
for i in range(num_merges):
    stats = get_stats(ids)
    pair = max(stats, key=stats.get)
    idx = 256 + i
    print(f"Fusionando {pair} -> {idx}")
    ids = merge(ids, pair, idx)
    merges[pair] = idx

Fusionando (101, 32) -> 256
Fusionando (110, 32) -> 257
Fusionando (100, 256) -> 258
Fusionando (97, 32) -> 259
Fusionando (32, 258) -> 260
Fusionando (117, 257) -> 261
Fusionando (97, 114) -> 262
Fusionando (260, 108) -> 263
Fusionando (97, 110) -> 264
Fusionando (99, 104) -> 265
Fusionando (256, 110) -> 266
Fusionando (256, 113) -> 267
Fusionando (267, 117) -> 268
Fusionando (105, 101) -> 269
Fusionando (69, 257) -> 270
Fusionando (270, 261) -> 271
Fusionando (271, 108) -> 272
Fusionando (272, 117) -> 273
Fusionando (273, 103) -> 274
Fusionando (274, 262) -> 275


### **2.5. Resultados de la Compresión**

Después de 20 fusiones, veamos cuánto hemos comprimido nuestra secuencia de texto original.

In [13]:
print("tokens length: ", len(text_bytes))
print("ids length: ", len(ids))
print(f"compression ratio: {len(text_bytes) / len(ids):.2f}X")

tokens length:  124
ids length:  79
compression ratio: 1.57X


## **3. Codificación y Decodificación**

Un tokenizador debe ser capaz de hacer dos cosas: **codificar** (texto -> tokens) y **decodificar** (tokens -> texto).

### **3.1. Decodificación**

Para decodificar, necesitamos un "vocabulario" que mapee cada ID de token (tanto los 256 originales como los nuevos que creamos) a su secuencia de bytes correspondiente. Lo construimos a partir de nuestras `merges`.

In [14]:
# Creamos el vocabulario inicial con los 256 bytes base.
vocab = {idx: bytes([idx]) for idx in range(256)}

# Añadimos los nuevos tokens fusionados. Cada nuevo token es la concatenación de los bytes de los dos tokens que lo formaron.
for (p0, p1), idx in merges.items():
    vocab[idx] = vocab[p0] + vocab[p1]

def decode(ids):
    # Recuperamos la secuencia de bytes completa concatenando los bytes de cada token.
    tokens = b"".join(vocab[idx] for idx in ids)
    
    # Decodificamos la secuencia de bytes a un string, reemplazando errores.
    # 'errors="replace"' es una salvaguarda importante: si el LLM genera una secuencia de bytes inválida en UTF-8, no se romperá, sino que mostrará un carácter de reemplazo '�'.
    text = tokens.decode("utf-8", errors="replace")
    return text

### **3.2. Codificación**

La codificación de un texto nuevo es el proceso de aplicar las fusiones aprendidas, **en el mismo orden en que se aprendieron**. Esto es crucial, ya que algunas fusiones dependen de otras anteriores.

La estrategia es:
1. Convertir el texto a su secuencia de bytes inicial.
2. Buscar en la secuencia todos los pares de tokens posibles.
3. De todos los pares que existen en nuestras `merges` aprendidas, aplicar el que se aprendió primero (el que tiene el ID más bajo).
4. Repetir hasta que no se puedan hacer más fusiones.

In [15]:
def encode(text):
    tokens = list(text.encode("utf-8"))
    while len(tokens) >= 2:
        stats = get_stats(tokens)
        # En lugar de buscar el par más frecuente, buscamos el par que aparece en nuestra lista de 'merges' con el índice más bajo.
        # merges.get(p, float("inf")) devuelve el índice de fusión para un par 'p', o infinito si el par nunca fue fusionado.
        # Al buscar el 'min', encontramos la fusión que se aprendió primero.
        pair = min(stats, key=lambda p: merges.get(p, float("inf")))
        # Si el par con el índice más bajo es 'inf', significa que no hay más pares fusionables en el texto.
        if pair not in merges:
            break # No hay nada más que fusionar
        idx = merges[pair]
        tokens = merge(tokens, pair, idx)
    return tokens

In [16]:
text = "Como estás?"

encoded = encode(text)
decoded = decode(encoded)

is_consistent = decode(encode(text)) == text
print(f"¿La decodificación de la codificación devuelve el texto original? -> {is_consistent}")

¿La decodificación de la codificación devuelve el texto original? -> True


# **4. Mejorando Nuestro Tokenizador: El Enfoque de GPT**

Nuestro tokenizador funciona, pero los tokenizadores de producción como los de la familia GPT introducen algunas mejoras clave.

### **4.1. Pre-tokenización con Expresiones Regulares (Regex)**

Si aplicamos BPE directamente sobre un texto, podríamos fusionar el final de una palabra con el principio de la siguiente:

Imagina que en nuestro texto de entrenamiento aparecen con mucha frecuencia palabras terminadas en "a" seguidas de un punto, como en "problema.", "casa." o "idea.".

El par de bytes para a. (la letra 'a' y el punto) se volvería muy frecuente. El algoritmo BPE, sin ninguna guía, crearía felizmente un nuevo token para a..

Ahora, cuando el tokenizador vea la palabra "Hola.", la podría dividir en:

`["Hol", "a."]` en lugar de la forma mucho más lógica `["Hola", "."]`.

**¿Por qué es esto un problema?**

1. **Pérdida de Información Gramatical**: El token . por sí solo tiene un significado muy fuerte: "fin de la oración". Al fusionarlo con "a", diluimos esa señal. El modelo tendría que aprender por separado el significado de a., o., s., etc., en lugar de aprender el concepto universal del punto final.
2. **Ineficiencia del Vocabulario**: Se generan tokens redundantes que mezclan conceptos. Es mucho más eficiente tener un token para "Hola" y otro para . que pueden combinarse de múltiples maneras.

Para evitarlo, GPT-2 y modelos posteriores usan una **pre-tokenización** basada en expresiones regulares. El texto se divide primero en "trozos" que suelen corresponder a palabras, puntuación o espacios. El algoritmo BPE se ejecuta de forma independiente dentro de cada uno de estos trozos.

In [17]:
import regex as re

gpt2pat = re.compile(r"""'s|'t|'re|'ve|'m|'ll|'d| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+""", re.IGNORECASE)

text = "Do you know where my 1st dog is?"
print(re.findall(gpt2pat, text))

['Do', ' you', ' know', ' where', ' my', ' 1', 'st', ' dog', ' is', '?']


Esto asegura que las fusiones respeten los límites de las palabras, generando tokens mucho más intuitivos. Pero, ¿cómo cambia exactamente el algoritmo BPE que implementamos antes?

La diferencia fundamental es el **ámbito de aplicación**. El algoritmo BPE ya no se ejecuta sobre una única y larga secuencia de bytes de todo el corpus. En su lugar, el proceso se modifica de la siguiente manera:

1.  **División Primero**: El texto completo se divide primero en una lista de "trozos" utilizando la expresión regular.
2.  **BPE Dentro de Cada Trozo**: El algoritmo BPE se ejecuta de forma **independiente y aislada dentro de cada uno de estos trozos**.

Esto significa que una fusión de bytes **nunca puede ocurrir a través de los límites de dos trozos diferentes**.

#### **Ejemplo Práctico: "coche azul"**

Veamos el impacto con un ejemplo sencillo:

*   **Sin Pre-tokenización (Método Antiguo):**
    *   El texto es una única secuencia de bytes: `b'coche azul'`.
    *   El algoritmo BPE podría analizar el par `(e,  )` (los bytes de la 'e' de "coche" y el espacio siguiente) y, si es muy frecuente en el corpus, fusionarlo. Esto es semánticamente indeseable.

*   **Con Pre-tokenización (Método GPT-2):**
    *   El regex divide el texto en una lista de trozos: `['coche', ' ', 'azul']`.
    *   El algoritmo BPE se ejecuta tres veces por separado:
        1.  **Sobre `b'coche'`: ** Buscará y fusionará los pares de bytes más comunes *solo dentro de esta palabra*.
        2.  **Sobre `b' '`: ** No hay pares que fusionar.
        3.  **Sobre `b'azul'`: ** Buscará y fusionará los pares de bytes más comunes *solo dentro de esta palabra*.

### **4.2. Tokens Especiales**

A menudo necesitamos tokens que no se derivan del texto, sino que actúan como instrucciones para el modelo. Ejemplos:
- `<|endoftext|>`: Indica el final de un documento.
- `<|im_start|>`: Marca el inicio de un mensaje en un chat.

Estos tokens se añaden al vocabulario **después** del entrenamiento de BPE con IDs reservados.

### **4.3. Un Vistazo a `tiktoken`**

`tiktoken` es la librería de tokenización de OpenAI. Es solo para inferencia (no se puede entrenar con ella), pero nos permite ver las diferencias entre tokenizadores, como el manejo de los espacios en GPT-2 vs GPT-4.

In [18]:
import tiktoken

# Texto de ejemplo con múltiples espacios al inicio
text_to_encode = "    Hello World"
print(f"Texto original: '{text_to_encode}'\n")

# --- Tokenizador de GPT-2 ---
# Este tokenizador tiende a tratar cada espacio como un token separado.
print("--- Tokenizador de GPT-2 (gpt2) ---")
enc_gpt2 = tiktoken.get_encoding("gpt2")
tokens_gpt2 = enc_gpt2.encode(text_to_encode)
print(f"Tokens: {tokens_gpt2}")
print(f"Número de tokens: {len(tokens_gpt2)}")


# --- Tokenizador de GPT-4 ---
# 'cl100k_base' es el tokenizador utilizado por modelos como GPT-3.5-turbo y GPT-4.
# Es más eficiente y ha aprendido a fusionar secuencias de espacios.
print("\n--- Tokenizador de GPT-4 (cl100k_base) ---")
enc_gpt4 = tiktoken.get_encoding("cl100k_base")
tokens_gpt4 = enc_gpt4.encode(text_to_encode)
print(f"Tokens: {tokens_gpt4}")
print(f"Número de tokens: {len(tokens_gpt4)}")

Texto original: '    Hello World'

--- Tokenizador de GPT-2 (gpt2) ---
Tokens: [220, 220, 220, 18435, 2159]
Número de tokens: 5

--- Tokenizador de GPT-4 (cl100k_base) ---
Tokens: [262, 22691, 4435]
Número de tokens: 3


Como vemos, GPT-4 aprendió a fusionar secuencias de espacios, mientras que GPT-2 las trata como tokens individuales. Estas decisiones de diseño afectan a la eficiencia y el comportamiento del modelo.

## **5. Alternativas: SentencePiece de Google**

`SentencePiece` es otra librería de tokenización muy popular, utilizada en modelos como Llama y Mistral. Permite tanto el entrenamiento como la inferencia.

Su principal diferencia filosófica con `tiktoken` es:
*   **tiktoken (GPT-style):** Trabaja directamente sobre la secuencia de **bytes UTF-8**. Es universal.
*   **SentencePiece:** Trabaja principalmente con **puntos de código Unicode**. No convierte a bytes a menos que encuentre un carácter desconocido, en cuyo caso utiliza un mecanismo de `byte_fallback`.

Vamos a entrenar un tokenizador `SentencePiece` básico.

In [ ]:
import sentencepiece as spm
import requests

URL = "https://github.com/ErikSarriegui/BPETokenizer/blob/main/text/toy_text.txt"
with open("text/toy_text.txt", "w") as file:
    file.write(requests.get(URL).text)

options = dict(
    # --- Input/Output ---
    input="text/toy_text.txt",                   # Fichero de texto para entrenar
    model_prefix="tok400",                  # Prefijo para los ficheros de salida (.model, .vocab)
    
    # --- Algoritmo ---
    model_type="bpe",                       # Usamos el algoritmo BPE
    vocab_size=400,                         # Tamaño del vocabulario deseado
    byte_fallback=True,                     # Si no conoce un carácter, lo descompone en sus bytes UTF-8
    
    # --- Normalización y Pre-tokenización ---
    normalization_rule_name="identity",     # 'identity' = no hacer normalización (ej: a NFC). Se recomienda no tocar el texto.
    remove_extra_whitespaces=False,         # No eliminar espacios extra
    split_digits=True,                      # Tratar cada dígito como un token separado al inicio
    add_dummy_prefix=True,                  # Añade un espacio al inicio de la frase para que " a" se tokenice igual que el " a" de "una casa".
    
    # --- Tokens Especiales ---
    unk_id=0,                               # ID para token desconocido (si byte_fallback=False)
    bos_id=1,                               # ID para "Beginning of Sentence"
    eos_id=2,                               # ID para "End of Sentence"
    pad_id=-1,                              # ID para "Padding" (ignorado, no se añade al vocabulario)
)

spm.SentencePieceTrainer.train(**options)

In [20]:
sp = spm.SentencePieceProcessor()
sp.load("tok400.model")
vocab = [[sp.id_to_piece(idx), idx ] for idx in range(sp.get_piece_size())]

texto_de_prueba = "más"
print(f"Texto a codificar: '{texto_de_prueba}'")

# Codificamos el texto en una secuencia de IDs numéricos.
ids = sp.encode(texto_de_prueba)
print(f"IDs resultantes: {ids}\n")

# Ahora, decodificamos los IDs de vuelta a sus "piezas" para inspeccionar qué hizo el tokenizador.
# Esta es la parte más instructiva.
piezas_decodificadas = [sp.id_to_piece(id) for id in ids]
print(f"Piezas decodificadas: {piezas_decodificadas}")

Texto a codificar: 'más'
IDs resultantes: [323, 350, 198, 164, 331]

Piezas decodificadas: ['▁', 'm', '<0xC3>', '<0xA1>', 's']


Observa cómo "más" se tokeniza. Como nuestro corpus de entrenamiento no contenía la letra "á", `SentencePiece` no tiene un token para ella. Gracias a `byte_fallback=True`, la descompone en sus dos bytes UTF-8. Sin esta opción, la habría reemplazado por el token `<unk>`.

## **6. Conclusiones Finales**

Hemos recorrido un largo camino, desde un simple `ord('h')` hasta entrenar tokenizadores con librerías estándar.

#### **Sobre el Tamaño del Vocabulario (`vocab_size`)**
Elegir el `vocab_size` es un compromiso:
*   **Vocabulario pequeño:**
    *   **Pros:** Menor coste computacional (capa de embeddings y de salida más pequeñas).
    *   **Contras:** Las secuencias de tokens son más largas, lo que aumenta el coste en el Transformer. Menos información por token.
*   **Vocabulario grande:**
    *   **Pros:** Secuencias de tokens más cortas (más eficiente para el Transformer). Tokens semánticamente más ricos.
    *   **Contras:** Más memoria y coste computacional en las capas de entrada/salida. Riesgo de que algunos tokens aparezcan tan poco que el modelo no aprenda bien su significado.

Los modelos actuales suelen usar vocabularios de decenas de miles (GPT-2: ~50k, Llama 3: 128k).

#### **Sobre la Multimodalidad**
¿Cómo manejan los modelos texto e imágenes? La clave está en la tokenización. No se cambia la arquitectura del Transformer, sino que se diseña un "tokenizador de imágenes" (normalmente una Red Neuronal Convolucional o un ViT) que convierte una imagen en una secuencia de vectores (tokens). Para el Transformer, son solo una secuencia de tokens más, indistinguibles de los que provienen del texto.

¡Gracias por seguir esta guía! Ahora tienes una base sólida para entender cómo los LLMs procesan el lenguaje.